<a href="https://colab.research.google.com/github/Davron030901/OpenCv/blob/main/26_Simple_Object_Tracking_by_Color.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Simple Object Tracking by Color**

####**In this lesson we'll learn:**
1. How to use an HSV Color Filter to Create a Mask and then Track our Desired Object


In [ ]:
# Our Setup, Import Libaries, Create our Imshow Function and Download our Images
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Define our imshow function
def imshow(title = "Image", image = None, size = 10):
    w, h = image.shape[0], image.shape[1]
    aspect_ratio = w/h
    plt.figure(figsize=(size * aspect_ratio,size))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()

!wget https://videos.pexels.com/video-files/4568863/4568863-hd_1920_1080_30fps.mp4

--2024-11-19 03:40:30--  https://videos.pexels.com/video-files/4568863/4568863-hd_1920_1080_30fps.mp4
Resolving videos.pexels.com (videos.pexels.com)... 104.18.67.220, 104.18.66.220, 2606:4700::6812:42dc, ...
Connecting to videos.pexels.com (videos.pexels.com)|104.18.67.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13294604 (13M)
Saving to: ‘4568863-hd_1920_1080_30fps.mp4’

4568863-hd_1920_108 100%[===================>]  12.68M  --.-KB/s    in 0.06s   

2024-11-19 03:40:30 (195 MB/s) - ‘4568863-hd_1920_1080_30fps.mp4’ saved [13294604/13294604]



In [ ]:
import cv2
import numpy as np

def init_video_capture(source):
    """Initialize video capture from file or camera"""
    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        raise Exception(f"Error: Could not open video source {source}")
    return cap

def init_video_writer(cap, output_filename):
    """Initialize video writer with same parameters as input video"""
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
    return cv2.VideoWriter(output_filename, fourcc, fps, (width, height))

def track_object(video_source, output_filename, lower_color, upper_color, min_radius=25):
    """
    Track colored object in video

    Parameters:
    video_source: int or str - Camera index or video file path
    output_filename: str - Output video file path
    lower_color: np.array - Lower HSV color bound
    upper_color: np.array - Upper HSV color bound
    min_radius: int - Minimum radius to track
    """
    # Initialize video capture
    cap = init_video_capture(video_source)

    # Initialize video writer
    out = init_video_writer(cap, output_filename)

    # Read first frame to get dimensions
    ret, frame = cap.read()
    if not ret:
        raise Exception("Could not read first frame")

    height, width = frame.shape[:2]
    points = []  # Store tracking points

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to HSV color space
        hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Create mask for color detection
        mask = cv2.inRange(hsv_img, lower_color, upper_color)

        # Find contours
        contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
                                     cv2.CHAIN_APPROX_SIMPLE)

        # Default center point
        center = (int(width/2), int(height/2))

        if contours:
            # Get largest contour
            c = max(contours, key=cv2.contourArea)
            (x, y), radius = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)

            try:
                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            except ZeroDivisionError:
                continue

            # Draw only if radius is large enough
            if radius > min_radius:
                cv2.circle(frame, (int(x), int(y)), int(radius), (0, 0, 255), 2)
                cv2.circle(frame, center, 5, (0, 255, 0), -1)
                points.append(center)

                # Draw tracking line
                for i in range(1, len(points)):
                    try:
                        cv2.line(frame, points[i - 1], points[i], (0, 255, 0), 2)
                    except:
                        pass

        out.write(frame)

        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage
if __name__ == "__main__":
    # Green color range in HSV
    lower_green = np.array([20, 50, 90])
    upper_green = np.array([40, 255, 255])

    # For video file
    try:
        track_object(
            video_source='/content/4568863-hd_1920_1080_30fps.mp4',  # Video fayl manzili
            output_filename='output.avi',
            lower_color=lower_green,
            upper_color=upper_green
        )
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
!ffmpeg -i /content/output.avi bmwm4_output.mp4 -y

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('bmwm4_output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

In [ ]:
HTML("""
<video controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)